# Day 3
-------
## Puzzle 1

We're given a 2d grid comprised of engine part numbers (digits), symbols, and `.` that denote empty spaces, a valid engine part number is any digit that is adjacent (horiz, vert, diag) from a symbol.

To solve this puzzle, we need to identify all valid engine part numbers in the schematic, and summ them to find the solution to the puzzle.

In [170]:
import numpy as np
from collections import defaultdict

Try pulling out the schematic into a numpy matrix, with the plan to just create a boolean mask of all valid positions. This way we can avoid overlap between symbols

In [120]:
def parse_schematic(inp):
    with open(f'data/{inp}') as f:
        schematic = [list(x.replace('\n','')) for x in f.readlines()]
    return np.array(schematic)

In [68]:
schem = parse_schematic('test.txt')

In [69]:
schem

array([['4', '6', '7', '.', '.', '1', '1', '4', '.', '.'],
       ['.', '.', '.', '*', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '3', '5', '.', '.', '6', '3', '3', '.'],
       ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.'],
       ['6', '1', '7', '*', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '+', '.', '5', '8', '.'],
       ['.', '.', '5', '9', '2', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '7', '5', '5', '.'],
       ['.', '.', '.', '$', '.', '*', '.', '.', '.', '.'],
       ['.', '6', '6', '4', '.', '5', '9', '8', '.', '.']], dtype='<U1')

Create the associated index mask

In [90]:
mask = np.zeros_like(schem, dtype=int)

In [108]:
shape = schem.shape

# Iterate through the elements of the schematic
for row, line in enumerate(schem):
    for col, val in enumerate(line):
        
        # Check that the current value is not a digit and not a '.'
        if not val.isdigit() and val != '.':

            # Loop through adjacent indices
            for i in np.arange(-1, 2):
                for j in np.arange(-1, 2):
                    row_mask = row+j if row+j >= 0 else 0
                    row_mask = row+j if row+j < shape[0] else shape[0]-1
                    col_mask = col+i if col+i >= 0 else 0
                    col_mask = col+i if col+i < shape[1] else shape[1]-1
                    
                    # If adjacent value is a digit then update the mask
                    if schem[row_mask, col_mask].isdigit():
#                         print(row_mask, col_mask)
                        mask[row_mask, col_mask] = True

In [107]:
schem[mask == True].astype(int).sum()

64

Well, I fucked this one up. What I get for trying to read the problem nwhile also watching the kids :D I originally thought that it was just the *digits* adjacent to the gears, not the whole numbers.

Let's start over. In this case, we probably want to iterate along rows, building up a number and at each digit, check if there is a gear adjacent to it. Once we reach a non-digit character or EOL, reset.

In [152]:
schem = parse_schematic('test.txt')
sum_val = 0

for row, line in enumerate(schem):
    num = 0
    part_num = False
    for col, val in enumerate(line):
        # check if the current value is a digit
        if val.isdigit():
            # increase the order of magnitude of the current number and add the cirrent value as the ones
            num = num * 10 + int(val)
            
            # Check if any of the surrounding values is a gear
            for i in np.arange(-1, 2):
                for j in np.arange(-1, 2):
                    row_mask = row+j if row+j >= 0 else 0
                    row_mask = row+j if row+j < shape[0] else shape[0]-1
                    col_mask = col+i if col+i >= 0 else 0
                    col_mask = col+i if col+i < shape[1] else shape[1]-1
                    
#                     print(f'{row_mask}, {col_mask}: {schem[row_mask, col_mask]}')
                    if not schem[row_mask, col_mask].isdigit() and schem[row_mask, col_mask] != '.':
                        # If it is a gear, mark it as a part number
#                         print(f"Found a gear: {not schem[row_mask, col_mask].isdigit()}, {schem[row_mask, col_mask] != '.'}")
                        part_num = True
        
        # check if the current value is not a digit or we are at the end of the row, then add the current number to the sum if it's a part number
        if not val.isdigit() or col == schem.shape[1] - 1:
            if part_num:
#                 print(f'Adding {num} to {sum_val}')
                sum_val += num
                part_num = False
            num = 0

In [153]:
# Check the test input answer
assert sum_val == 4361

In [162]:
schem = parse_schematic('input.txt')
sum_val = 0

for row, line in enumerate(schem):
    num = 0
    part_num = False
    for col, val in enumerate(line):
        # check if the current value is a digit
        if val.isdigit():
            # increase the order of magnitude of the current number and add the cirrent value as the ones
            num = num * 10 + int(val)
#             print(num)
            # Check if any of the surrounding values is a gear
            for i in np.arange(-1, 2):
                for j in np.arange(-1, 2):
                    row_mask = row+i
                    if row_mask < 0:
                        row_mask = 0
                    elif row_mask >= schem.shape[0]:
                        row_mask = schem.shape[0] - 1
                        
                    col_mask = col+j
                    if col_mask < 0:
                        col_mask = 0
                    elif col_mask >= schem.shape[1]:
                        col_mask = schem.shape[1] - 1
                
                    
#                     print(f'{row_mask}, {col_mask}: {schem[row_mask, col_mask]}')
                    if not schem[row_mask, col_mask].isdigit() and schem[row_mask, col_mask] != '.':
                        # If it is a gear, mark it as a part number
#                         print(f"Found a gear: {not schem[row_mask, col_mask].isdigit()}, {schem[row_mask, col_mask] != '.'}")
                        part_num = True
        
        # check if the current value is not a digit or we are at the end of the row, then add the current number to the sum if it's a part number
        if not val.isdigit() or col == schem.shape[1] - 1:
            if part_num:
#                 print(f'Adding {num} to {sum_val}')
                sum_val += num
                part_num = False
            num = 0

In [163]:
sum_val

529618

### Success!!
---
## Puzzle 2

Now we need to find specific gears. A gear is any `*` character that is adjacent to *exactly two part numbers*. For those gears, calculate the product of the two gears and sum all ratios together for the answer

We'll need to track when a number finds an adjacent `*` and then for that index keep a list of the adjacent part numbers. Once we reach the end of a part number, check if the list for that gear is equal to 2 and then get the product

We're going to make a dumb assumption that each number is only adjacent to no more than one `*` and hope that's correct

In [179]:
schem = parse_schematic('input.txt')

gear_lists = defaultdict(list)
for row, line in enumerate(schem):
    num = 0
    gear = None
    for col, val in enumerate(line):
        # check if the current value is a digit
        if val.isdigit():
            # increase the order of magnitude of the current number and add the cirrent value as the ones
            num = num * 10 + int(val)
#             print(num)
            # Check if any of the surrounding values is a gear
            for i in np.arange(-1, 2):
                for j in np.arange(-1, 2):
                    row_mask = row+i
                    if row_mask < 0:
                        row_mask = 0
                    elif row_mask >= schem.shape[0]:
                        row_mask = schem.shape[0] - 1
                        
                    col_mask = col+j
                    if col_mask < 0:
                        col_mask = 0
                    elif col_mask >= schem.shape[1]:
                        col_mask = schem.shape[1] - 1
                
                    
#                     print(f'{row_mask}, {col_mask}: {schem[row_mask, col_mask]}')
                    if schem[row_mask, col_mask] == '*':
                        # If it is a gear, mark it as a part number
#                         print(f"Found a gear: {not schem[row_mask, col_mask].isdigit()}, {schem[row_mask, col_mask] != '.'}")
                        gear = (row_mask, col_mask)
        
        # check if the current value is not a digit or we are at the end of the row, then add the current number to the sum if it's a part number
        if not val.isdigit() or col == schem.shape[1] - 1:
            if num > 0 and gear is not None:
                gear_lists[gear].append(num)

            gear = None
            num = 0

In [183]:
sum_val = 0
for loc, parts in gear_lists.items():
    if len(parts) == 2:
        sum_val += (parts[0] * parts[1])

In [184]:
sum_val

77509019

## SUCCESS! Day 3 finished

Got lucky with that assumption in the second part

In [ ]:
!git add *
!git commit -m "Day 2 solution"
!git push -u origin main